In [1]:
import sys
from sklearn.model_selection import train_test_split
import numpy as np
import logging
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
#import graphviz
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.utils import shuffle
from sklearn.model_selection import LeaveOneOut
from xgboost import XGBClassifier

In [ ]:
lead_time = [6,12,24,48]

classifiers = [
    KNeighborsClassifier(n_jobs=1),
    linear_model.LogisticRegression(C=1e5),
    svm.SVC(C=1800.0, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',max_iter=-1, probability=True, random_state=13, shrinking=True,tol=0.001, verbose=False),
    tree.DecisionTreeClassifier(),
    GaussianNB(),
    AdaBoostClassifier(n_estimators=600,base_estimator=tree.DecisionTreeClassifier(max_depth=2)),
    XGBClassifier(n_estimators=600,silent=True, nthread=8, max_depth=7, scale_pos_weight=3.5),
    RandomForestClassifier(max_depth=8, n_estimators=2000, n_jobs=4),
    MLPClassifier(hidden_layer_sizes=(50,), max_iter=500, alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=1,
                    learning_rate_init=.1),
    QuadraticDiscriminantAnalysis()
]
cls_names=[
    'KNN',
    'Logist',
    'SVM',
    'DTree',
    'NB',
    'ADA',
    'xgboost',
    'RFs',
    'MLP',
    'QDA'
]

for cls, cls_name in zip(classifiers, cls_names):
    print(cls_name)
    for t in lead_time:
        # for global tropic
        pos_x    = np.loadtxt("data/new/positive_global_MCS_TC_GP_features_"+str(t)+"hours.txt")
        neg_x    = np.loadtxt("data/new/negative_global_MCS_TC_GP_features.txt")
        # for NA
        #pos_x    = np.loadtxt("data/new/positive_NA_MCS_TC_GP_features_"+str(t)+"hours.txt")
        #neg_x    = np.loadtxt("data/new/negative_NA_MCS_TC_GP_features.txt")
        # for WNP
        #pos_x    = np.loadtxt("data/new/positive_WNP_MCS_TC_GP_features_"+str(t)+"hours.txt")
        #neg_x    = np.loadtxt("data/new/negative_WNP_MCS_TC_GP_features.txt")
       
        print(t)
        
        pos_scl = MinMaxScaler()
        neg_scl = MinMaxScaler()
        pos_scl.fit(pos_x)  
        neg_scl.fit(neg_x)
        pos_x_scl = pos_scl.transform(pos_x)
        neg_x_scl = neg_scl.transform(neg_x)

        pos_len  = pos_x.shape[0]
        neg_len  = neg_x.shape[0]

        pos_y = np.empty(pos_len)
        neg_y = np.empty(neg_len)
        pos_y.fill(1)
        neg_y.fill(0)

        MCS_x = np.concatenate((pos_x_scl,neg_x_scl),axis=0)
        MCS_y = np.concatenate((pos_y,neg_y),axis=0)
    
        MCS_x, MCS_y = shuffle(MCS_x, MCS_y, random_state=13)
        #MCS_x_train,MCS_x_test,MCS_y_train,MCS_y_test= train_test_split(MCS_x, MCS_y, test_size=0.2, random_state=10)
        cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
        
        f1_scores = cross_val_score(cls, MCS_x, MCS_y, cv=cv, scoring='f1_macro')
        p_scores = cross_val_score(cls, MCS_x, MCS_y, cv=cv, scoring='precision_macro')
        r_scores = cross_val_score(cls, MCS_x, MCS_y, cv=cv, scoring='recall_macro')
        print('{:.03f}'.format(f1_scores.mean()))
        print('{:.03f}'.format(p_scores.mean()))
        print('{:.03f}'.format(r_scores.mean()))

KNN
6
0.835
0.885
0.801
12
0.803
0.866
0.763
24
0.770
0.913
0.706
48


/global/common/cori/software/python/3.6-anaconda-5.2/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/global/common/cori/software/python/3.6-anaconda-5.2/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/global/common/cori/software/python/3.6-anaconda-5.2/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/global/common/cori/software/python/3.6-anaconda-5.2/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being

0.580
0.762
0.552
Logist
6
0.778
0.841
0.741
12
0.765
0.846
0.721
24
0.774
0.823
0.742
48
0.680
0.763
0.640
SVM
6
0.910
0.942
0.883
12
0.867
0.912
0.834
24
0.889
0.923
0.861
48
0.844
0.950
0.781
DTree
6
0.835
0.830
0.830
12
0.786
0.781
0.793
24
0.812
0.810
0.822
48
0.694
0.697
0.678
NB
6
0.763
0.765
0.761
12
0.739
0.729
0.753
24
0.722
0.693
0.767
48
0.595
0.565
0.730
ADA
6
0.969
0.979
0.959
12
0.955
0.968
0.943
24
0.954
0.967
0.942
48
0.843
0.975
0.797
xgboost
6
0.964
0.970
0.959
12
0.937
0.950
0.926
24
0.930
0.945
0.916
48
0.821
0.936
0.757
RFs
6
0.865
0.912
0.831
12
0.832
0.921
0.780
24
0.839
0.919
0.787
48
